# CTDCAL

### Import Libraries

In [ ]:
import os
import time
import sys
sys.path.append('ctdcal/')
import settings
import ctdcal.process_ctd as process_ctd
import ctdcal.fit_ctd as fit_ctd
import pandas as pd
import numpy as np
import gsw
import ctdcal.oxy_fitting as oxy_fitting
import sbe_convert
import matplotlib.pyplot as plt
from decimal import Decimal
plt.style.use('dark_background')
import scipy
import rinko

### Get Variables from settings.py

In [ ]:
    # Directory and file information
    expocode = settings.cruise['expocode']
    sectionID = settings.cruise['sectionid']
    raw_directory = settings.ctd_processing_dir['raw_data_directory']
    time_directory = settings.ctd_processing_dir['time_data_directory']
    converted_directory = settings.ctd_processing_dir['converted_directory']
    pressure_directory = settings.ctd_processing_dir['pressure_data_directory']
    oxygen_directory = settings.ctd_processing_dir['oxygen_directory']
    btl_directory = settings.ctd_processing_dir['bottle_directory']
    o2flask_file = settings.ctd_processing_dir['o2flask_file']
    log_directory = settings.ctd_processing_dir['log_directory']
    p_log_file = settings.ctd_processing_dir['pressure_log']
    hex_prefix = settings.ctd_processing_dir['hex_prefix']
    hex_postfix = settings.ctd_processing_dir['hex_postfix']
    xml_prefix = settings.ctd_processing_dir['xml_prefix']
    xml_postfix = settings.ctd_processing_dir['xml_postfix']
    
    # CTD Data Inputs
    p_col = settings.ctd_inputs['p']
    t_col = settings.ctd_inputs['t2']
    t1_col = settings.ctd_inputs['t1']
    t2_col = settings.ctd_inputs['t2']
    c_col = settings.ctd_inputs['c2']
    c1_col = settings.ctd_inputs['c1']
    c2_col = settings.ctd_inputs['c2']
    sal_col = settings.ctd_inputs['salt']
    dov_col = settings.ctd_inputs['dov']
    lat_col = settings.ctd_inputs['lat']
    lon_col = settings.ctd_inputs['lon']
    time_col = settings.ctd_inputs['scan_datetime']
    rinko_volts = settings.ctd_inputs['rinko_oxy']
    

        # Bottle Data Inputs
    p_btl_col = settings.bottle_inputs['p']
    t_btl_col = settings.bottle_inputs['t2']
    t1_btl_col = settings.bottle_inputs['t1']
    t2_btl_col = settings.bottle_inputs['t2']
    c_btl_col = settings.bottle_inputs['c2']
    c1_btl_col = settings.bottle_inputs['c1']
    c2_btl_col = settings.bottle_inputs['c2']
    reft_col = settings.bottle_inputs['reft']
    cond_col = settings.bottle_inputs['btl_cond']
    cr_avg = settings.bottle_inputs['cond_ratio']
    bath_temp = settings.bottle_inputs['bath_temp']
    sal_btl_col = settings.bottle_inputs['salt']
    dov_btl_col = settings.bottle_inputs['dov']
    lat_btl_col = settings.bottle_inputs['lat']
    lon_btl_col = settings.bottle_inputs['lon']
    oxy_btl_col = settings.bottle_inputs['btl_oxy']
    time_btl_col = settings.bottle_inputs['scan_datetime']
    btl_num = settings.bottle_inputs['btl_num']
    rinko_btl_volts = settings.bottle_inputs['rinko_oxy']
    
    # CTD Information    
    sample_rate = settings.ctd_processing_constants['sample_rate']
    search_time = settings.ctd_processing_constants['roll_filter_time']
    ctd = settings.ctd_processing_constants['ctd_serial']
    
    p_column_names = settings.pressure_series_output['column_names']
    p_column_units = settings.pressure_series_output['column_units']
    
    btl_data_prefix = 'data/bottle/'
    btl_data_postfix = '_btl_mean.pkl'
    time_data_prefix = 'data/time/'
    time_data_postfix = '_time.pkl'
    p_log_file = 'data/logs/ondeck_pressure.csv'
 

    
    # Columns from btl and ctd file to be read:
    btl_cols = settings.btl_input_array
    ctd_cols = settings.ctd_input_array
    
    ssscc = settings.ssscc

#    time_start = time.perf_counter()
    cnv_dir_list = os.listdir(converted_directory)
    time_dir_list = os.listdir(time_directory)
    btl_dir_list = os.listdir(btl_directory)

### Get last cast from each station (optional)

In [ ]:
# import numpy as np
# ssscc_2 = []
# for x in ssscc:
#     ssscc_2.append(x[0:3])
    
# arr, index, counts = np.unique(ssscc_2,return_index=True,return_counts=True)
# last_instance = counts - 1
# last_instance = list(last_instance + index)
# ssscc = np.array(ssscc)
# ssscc = list(ssscc[last_instance])



### Remove any specific stations (optional)

In [ ]:
ssscc.remove('90101')
ssscc.remove('00901')

In [ ]:
ssscc = ssscc[78:88]

### _Convert SBE files to pkl_

In [ ]:
    for station in ssscc:
        if '{}.pkl'.format(station) in cnv_dir_list:
            continue
        #convert hex to ctd
        hex_file = hex_prefix + station + hex_postfix
        xml_file = xml_prefix + station + xml_postfix
        
        sbe_convert.convert_sbe(station, hex_file, xml_file, converted_directory)
        print('Converted_sbe SSSCC: ' + station + ' done')

### _Create Time files_

In [ ]:
    for station in ssscc:
        if '{}_time.pkl'.format(station) in time_dir_list:
            continue
        sbe_convert.sbe_metadata(station)
        print('sbe_metadata SSSCC: ' + station + ' done')

### _Create Bottle files_

In [ ]:
    for station in ssscc:
        if '{}_btl_mean.pkl'.format(station) in btl_dir_list:
            continue
        #process bottle file
        sbe_convert.process_bottle(station)
        print('process_bottle SSSCC: ' + station + ' done')

### _Load all btl and ctd files_

In [ ]:
    btl_data_all = process_ctd.load_all_ctd_files(ssscc,btl_data_prefix,
                                                  btl_data_postfix,'bottle',btl_cols)
    time_data_all = process_ctd.load_all_ctd_files(ssscc,time_data_prefix,
                                                   time_data_postfix,'time',ctd_cols)
    btl_data_all.drop_duplicates(inplace=True)
    
    # Add Expocode and sect_id columns
    btl_data_all['EXPOCODE'] = settings.cruise['expocode']
    btl_data_all['SECT_ID'] = settings.cruise['sectionid']
    # Add Station and Cast number columns from ssscc column
    btl_data_all['STNNBR'] = process_ctd.stnnbr_from_ssscc(btl_data_all['SSSCC'])
    btl_data_all['CASTNO'] = process_ctd.castno_from_ssscc(btl_data_all['SSSCC'])
    # Add btl number and sample number
    btl_data_all = process_ctd.add_btlnbr_cols(btl_data_all, btl_num)
    btl_data_all = process_ctd.add_sampno_col(btl_data_all, btl_num)

# Pressure Calibration

In [ ]:
    pressure_log = process_ctd.load_pressure_logs(p_log_file)
    p_off = process_ctd.get_pressure_offset(pressure_log.ondeck_start_p,pressure_log.ondeck_end_p)
    
    if ~np.isnan(p_off):
        btl_data_all = fit_ctd.apply_pressure_offset(btl_data_all, p_btl_col, p_off)
        time_data_all = fit_ctd.apply_pressure_offset(time_data_all, p_btl_col, p_off)

# Temperature Calibration

In [ ]:
    df_ques_t1 = pd.DataFrame()
    df_ques_t2 = pd.DataFrame()
    
    df_ques_c1 = pd.DataFrame()
    df_ques_c2 = pd.DataFrame()

    ### Temperature Calibration   
    for x in range(2):
        
     # Second order calibration
             
        df_temp_good = process_ctd.prepare_fit_data(btl_data_all, reft_col)
        
        df_ques_reft = process_ctd.quality_check(df_temp_good[t2_btl_col], df_temp_good[t1_btl_col], df_temp_good[p_btl_col], df_temp_good['SSSCC'], df_temp_good['btl_fire_num'], 'quest')
        df_ques_reft['Parameter'] = 'REF_TEMP'
        


        if settings.do_primary == 1:
            coef_temp_1,df_ques_t1 = process_ctd.calibrate_param(df_temp_good[t1_btl_col], df_temp_good[reft_col], df_temp_good[p_btl_col], 'TP', 2, df_temp_good.SSSCC, df_temp_good.btl_fire_num, xRange='800:6000')
            btl_data_all[t1_btl_col] = fit_ctd.temperature_polyfit(btl_data_all[t1_btl_col], btl_data_all[p_btl_col], coef_temp_1)
            time_data_all[t1_col] = fit_ctd.temperature_polyfit(time_data_all[t1_col], time_data_all[p_col], coef_temp_1)
        
        if settings.do_secondary == 1:
            coef_temp_2,df_ques_t2 = process_ctd.calibrate_param(df_temp_good[t2_btl_col], df_temp_good[reft_col], df_temp_good[p_btl_col], 'TP', 2, df_temp_good.SSSCC, df_temp_good.btl_fire_num, xRange='1500:6000')
            btl_data_all[t2_btl_col] = fit_ctd.temperature_polyfit(btl_data_all[t2_btl_col], btl_data_all[p_btl_col], coef_temp_2)
            time_data_all[t2_col] = fit_ctd.temperature_polyfit(time_data_all[t2_col], time_data_all[p_col], coef_temp_2)
    
    # Apply fitting coef to data
  
    # Construct Quality Flag file
    
        qual_flag_temp = process_ctd.combine_quality_flags([df_ques_reft,df_ques_t1,df_ques_t2])
   
    ## First order calibtation
    
        df_temp_good = process_ctd.prepare_fit_data(btl_data_all, reft_col)
        
#        df_ques_reft = process_ctd.quality_check(df_temp_good[t2_btl_col], df_temp_good[t1_btl_col], df_temp_good[p_btl_col], df_temp_good['SSSCC'], df_temp_good['btl_fire_num'], 'quest')
#        df_ques_reft['Parameter'] = 'REF_TEMP'
        if settings.do_primary == 1:
            coef_temp_prim,df_ques_t1 = process_ctd.calibrate_param(df_temp_good[t1_btl_col], df_temp_good[reft_col], df_temp_good[p_btl_col], 'T', 1, df_temp_good.SSSCC, df_temp_good.btl_fire_num)
            btl_data_all[t1_btl_col] = fit_ctd.temperature_polyfit(btl_data_all[t1_btl_col], btl_data_all[p_btl_col], coef_temp_prim)
            time_data_all[t1_col] = fit_ctd.temperature_polyfit(time_data_all[t1_col], time_data_all[p_col], coef_temp_prim)
        
        if settings.do_secondary == 1:
            coef_temp_sec,df_ques_t2 = process_ctd.calibrate_param(df_temp_good[t2_btl_col], df_temp_good[reft_col], df_temp_good[p_btl_col], 'T', 1, df_temp_good.SSSCC, df_temp_good.btl_fire_num)
            btl_data_all[t2_btl_col] = fit_ctd.temperature_polyfit(btl_data_all[t2_btl_col], btl_data_all[p_btl_col], coef_temp_sec)
            time_data_all[t2_col] = fit_ctd.temperature_polyfit(time_data_all[t2_col], time_data_all[p_col], coef_temp_sec)
        
        
        
    # Apply fitting coef to data
            
        qual_flag_temp = process_ctd.combine_quality_flags([df_ques_reft,df_ques_t1,df_ques_t2])
    time_data_all['CTDTMP'] = time_data_all[t_col]
    time_data_all['CTDTMP_FLAG_W'] = 2
    btl_data_all['CTDTMP'] = btl_data_all[t_btl_col]

# Conductivity Calibration

In [ ]:
    for x in range(2):
        
        btl_data_all[cond_col] = fit_ctd.CR_to_cond(btl_data_all[cr_avg], btl_data_all[bath_temp], btl_data_all[t1_btl_col], btl_data_all[p_btl_col])
        df_cond_good = process_ctd.prepare_fit_data(btl_data_all, cond_col)

        df_ques_refc = process_ctd.quality_check(df_cond_good[c2_btl_col], df_cond_good[c1_btl_col], df_cond_good[p_btl_col], df_cond_good['SSSCC'], df_cond_good['btl_fire_num'], 'quest')
        df_ques_refc['Parameter'] = 'REF_COND'

    # Second Order Calibration
        if settings.do_primary == 1:
            coef_cond_1,df_ques_c1 = process_ctd.calibrate_param(df_cond_good[c1_btl_col], df_cond_good[cond_col], df_cond_good[p_btl_col], 'CP', 2, df_cond_good['SSSCC'], df_cond_good['btl_fire_num'], xRange='800:6000')
            btl_data_all[c1_btl_col] = fit_ctd.conductivity_polyfit(btl_data_all[c1_btl_col], btl_data_all[t1_btl_col], btl_data_all[p_btl_col], coef_cond_1)
            time_data_all[c1_col] = fit_ctd.conductivity_polyfit(time_data_all[c1_col], time_data_all[t1_col], time_data_all[p_col], coef_cond_1)
        
        if settings.do_secondary == 1:
            coef_cond_2,df_ques_c2 = process_ctd.calibrate_param(df_cond_good[c2_btl_col], df_cond_good[cond_col], df_cond_good[p_btl_col], 'CP', 2, df_cond_good['SSSCC'], df_cond_good['btl_fire_num'], xRange='1500:6000')
            btl_data_all[c2_btl_col] = fit_ctd.conductivity_polyfit(btl_data_all[c2_btl_col], btl_data_all[t2_btl_col], btl_data_all[p_btl_col] ,coef_cond_2)
            time_data_all[c2_btl_col] = fit_ctd.conductivity_polyfit(time_data_all[c2_col], time_data_all[t2_col], time_data_all[p_col], coef_cond_2)

        qual_flag_cond = process_ctd.combine_quality_flags([df_ques_c1,df_ques_c2,df_ques_refc])
    
        
        btl_data_all[cond_col] = fit_ctd.CR_to_cond(btl_data_all[cr_avg], btl_data_all[bath_temp], btl_data_all[t1_btl_col], btl_data_all[p_btl_col])
        df_cond_good = process_ctd.prepare_fit_data(btl_data_all,cond_col)
    
        if settings.do_primary == 1:
            coef_cond_prim,df_ques_c1 = process_ctd.calibrate_param(df_cond_good[c1_btl_col], df_cond_good[cond_col], df_cond_good[p_btl_col], 'C', 2 , df_cond_good['SSSCC'], df_cond_good['btl_fire_num'])
            btl_data_all[c1_btl_col] = fit_ctd.conductivity_polyfit(btl_data_all[c1_btl_col], btl_data_all[t1_btl_col], btl_data_all[p_btl_col], coef_cond_prim)        
            time_data_all[c1_col] = fit_ctd.conductivity_polyfit(time_data_all[c1_col], time_data_all[t1_col], time_data_all[p_col], coef_cond_prim)
        
        if settings.do_secondary == 1:
            coef_cond_sec,df_ques_c2 = process_ctd.calibrate_param(df_cond_good.CTDCOND2,df_cond_good.BTLCOND,df_cond_good.CTDPRS,'C',2,df_cond_good.SSSCC,df_cond_good.btl_fire_num)
            btl_data_all[c2_btl_col] = fit_ctd.conductivity_polyfit(btl_data_all[c2_btl_col], btl_data_all[t2_btl_col], btl_data_all[p_btl_col], coef_cond_sec)
            time_data_all[c2_col] = fit_ctd.conductivity_polyfit(time_data_all[c2_col], time_data_all[t2_col], time_data_all[p_col], coef_cond_sec)
        
    
        qual_flag_cond = process_ctd.combine_quality_flags([df_ques_c1, df_ques_c2, df_ques_refc])
        
    btl_data_all[sal_btl_col] = gsw.SP_from_C(btl_data_all[c_btl_col],btl_data_all[t_btl_col],btl_data_all[p_btl_col])
    time_data_all[sal_col] = gsw.SP_from_C(time_data_all[c_col], time_data_all[t_col], time_data_all[p_col])
    time_data_all['CTDSAL_FLAG_W'] = 2

# **Oxygen Calibration**

### *Calculate required oxygen parameter*

In [ ]:
    # Calculate Sigma
    btl_data_all['sigma_btl'] = oxy_fitting.sigma_from_CTD(btl_data_all[sal_btl_col], btl_data_all[t_btl_col], btl_data_all[p_btl_col], btl_data_all[lon_btl_col], btl_data_all[lat_btl_col])
    time_data_all['sigma_ctd'] = oxy_fitting.sigma_from_CTD(time_data_all[sal_col], time_data_all[t_col], time_data_all[p_col], time_data_all[lon_col], time_data_all[lat_col])

    btl_data_all[oxy_btl_col] = oxy_fitting.calculate_bottle_oxygen(ssscc, btl_data_all['SSSCC'], btl_data_all['TITR_VOL'], btl_data_all['TITR_TEMP'], btl_data_all['FLASKNO'])
    btl_data_all[oxy_btl_col] = oxy_fitting.oxy_ml_to_umolkg(btl_data_all[oxy_btl_col], btl_data_all['sigma_btl'])
    btl_data_all['OXYGEN_FLAG_W'] = oxy_fitting.flag_winkler_oxygen(btl_data_all[oxy_btl_col])

    # Calculate SA and PT
    btl_data_all['SA'] = gsw.SA_from_SP(btl_data_all[sal_btl_col], btl_data_all[p_btl_col], btl_data_all[lon_btl_col], btl_data_all[lat_btl_col])
    btl_data_all['PT'] = gsw.pt0_from_t(btl_data_all['SA'], btl_data_all[t_btl_col], btl_data_all[p_btl_col])

    time_data_all['SA'] = gsw.SA_from_SP(time_data_all[sal_col], time_data_all[p_col], time_data_all[lon_col], time_data_all[lat_col])
    time_data_all['PT'] = gsw.pt0_from_t(time_data_all['SA'], time_data_all[t_col], time_data_all[p_col])

    # Calculate OS in µmol/kg

    btl_data_all['OS_btl'] = oxy_fitting.os_umol_kg(btl_data_all['SA'], btl_data_all['PT'])
    time_data_all['OS_ctd'] = oxy_fitting.os_umol_kg(time_data_all['SA'], time_data_all['PT'])   

    coef_dict = {}

In [ ]:
time_data_all.sort_values(by='sigma_ctd',inplace=True)
btl_data_oxy = btl_data_all[btl_data_all['OXYGEN'].notna()].copy()
btl_data_oxy.sort_values(by='sigma_btl',inplace=True)

### *Collect data by station*

In [ ]:
btl_data_all['oxy_stn_group'] = btl_data_all['SSSCC'].str[0:3]
time_data_all['oxy_stn_group'] = time_data_all['SSSCC'].str[0:3]

In [ ]:
station_list = time_data_all['oxy_stn_group'].unique()
station_list = station_list.tolist()
station_list.sort()

In [ ]:
btl_data_oxy = btl_data_all.copy()#loc[btl_data_all['OXYGEN'].notnull()]

In [ ]:
rinko_coef0 = rinko.rinko_o2_cal_parameters()


In [ ]:
all_rinko_df = pd.DataFrame()
all_sbe43_df = pd.DataFrame()
rinko_dict = {}
sbe43_dict = {}
for station in station_list:
    
    #time_data = time_data_all[time_data_all['SSSCC'].str[0:3] == station].copy()
    #btl_data = btl_data_oxy[btl_data_oxy['SSSCC'].str[0:3] == station].copy()
    
    time_data = time_data_all[time_data_all['oxy_stn_group'] == station].copy()
    btl_data = btl_data_oxy[btl_data_oxy['oxy_stn_group'] == station].copy()
    
    rinko_coef, rinko_oxy_df = rinko.rinko_oxygen_fit(btl_data[p_btl_col],btl_data[oxy_btl_col],btl_data['sigma_btl'],time_data['sigma_ctd'],
                           time_data['OS_ctd'],time_data[p_col],time_data[t_col],time_data[rinko_volts],rinko_coef0, btl_data['SSSCC']
            )
    station_ssscc = time_data['SSSCC'].values[0]
    hex_file = hex_prefix + station_ssscc + hex_postfix
    xml_file = xml_prefix + station_ssscc + xml_postfix
    sbe_coef0 = oxy_fitting.get_SB_coef(hex_file, xml_file)
    sbe_coef, sbe_oxy_df = oxy_fitting.sbe43_oxy_fit(btl_data[p_btl_col], btl_data[oxy_btl_col], btl_data['sigma_btl'],time_data['sigma_ctd'],
                                                     time_data['OS_ctd'],time_data[p_col],time_data[t_col],time_data[dov_col],
                                                     time_data['scan_datetime'],sbe_coef0,btl_data['SSSCC']
                                                    )
    
    
    rinko_dict[station] = rinko_coef
    sbe43_dict[station] = sbe_coef
    all_rinko_df = pd.concat([all_rinko_df,rinko_oxy_df])
    all_sbe43_df = pd.concat([all_sbe43_df,sbe_oxy_df])
    
    print(station + ' Done!')

sbe43_coef_df = oxy_fitting.create_coef_df(sbe43_dict)
rinko_coef_df = oxy_fitting.create_coef_df(rinko_dict)

btl_data_all = btl_data_all.merge(all_rinko_df, left_on=['SSSCC',p_btl_col], right_on=['SSSCC_rinko','CTDPRS_rinko_btl'],how='left')

btl_data_all = btl_data_all.merge(all_sbe43_df, left_on=['SSSCC',p_btl_col], right_on=['SSSCC_sbe43','CTDPRS_sbe43_btl'],how='left')

btl_data_all.drop(list(btl_data_all.filter(regex = 'rinko')), axis = 1, inplace = True)

btl_data_all.drop(list(btl_data_all.filter(regex = 'sbe43')), axis = 1, inplace = True)

### Handle Missing Values

X = btl_data_all['CTDOXYVOLTS_x'], btl_data_all['CTDPRS'], btl_data_all['CTDTMP'], btl_data_all['dv_dt_x'], btl_data_all['OS_btl']
rinko_X = btl_data_all[p_btl_col], btl_data_all[t_btl_col], btl_data_all[rinko_btl_volts], btl_data_all['OS_btl']

for station in btl_data_all['SSSCC']:
    coef_43 = sbe43_coef_df.loc[station[0:3]]
    coef_rinko = rinko_coef_df.loc[station[0:3]]
    btl_data_all['CTDOXY_fill'] = oxy_fitting.oxy_equation(X, coef_43[0], coef_43[1], coef_43[2], coef_43[3], coef_43[4], coef_43[5], coef_43[6])
    btl_data_all['CTDRINKO_fill'] = rinko.rinko_curve_fit_eq(rinko_X, coef_rinko[0], coef_rinko[1], coef_rinko[2], coef_rinko[3], coef_rinko[4], coef_rinko[5], coef_rinko[6], coef_rinko[7])

btl_data_all.loc[btl_data_all['CTDOXY'].isnull(),'CTDOXY_FLAG_W'] = 2
btl_data_all.loc[btl_data_all['CTDOXY'].isnull(),'CTDOXY'] = btl_data_all.loc[btl_data_all['CTDOXY'].isnull(),'CTDOXY_fill']
btl_data_all.loc[btl_data_all['CTDRINKO'].isnull(),'CTDRINKO_FLAG_W'] = 2
btl_data_all.loc[btl_data_all['CTDRINKO'].isnull(),'CTDRINKO'] = btl_data_all.loc[btl_data_all['CTDRINKO'].isnull(),'CTDRINKO_fill']

In [ ]:
btl_data_all = oxy_fitting.flag_oxy_data(btl_data_all)
btl_data_all = oxy_fitting.flag_oxy_data(btl_data_all,ctd_oxy_col='CTDRINKO',flag_col='CTDRINKO_FLAG_W')

In [ ]:
btl_data_all['res_rinko'] = btl_data_all['OXYGEN'] - btl_data_all['CTDRINKO']
btl_data_all['res_sbe43'] = btl_data_all['OXYGEN'] - btl_data_all['CTDOXY']
btl_data_all.loc[np.abs(btl_data_all['res_rinko']) >=6 , 'CTDRINKO_FLAG_W'] = 3
btl_data_all.loc[np.abs(btl_data_all['res_sbe43']) >=6 , 'CTDOXY_FLAG_W'] = 3

In [ ]:
good_df = btl_data_all[btl_data_all['CTDRINKO_FLAG_W'] == 2]
print(good_df['res_rinko'].abs().std())
print(len(good_df))
plt.plot(good_df['res_rinko'], -good_df['CTDPRS'],'x')
plt.xlim(-10,10)

In [ ]:
good_df = btl_data_all[btl_data_all['CTDOXY_FLAG_W'] == 2]
print(good_df['res_sbe43'].abs().std())
print(len(good_df))
plt.plot(good_df['res_sbe43'], -good_df['CTDPRS'],'x')
plt.xlim(-10,10)

In [ ]:
time_data_all['CTDOXY'] = '-999'
time_data_all['CTDRINKO'] = '-999'
btl_data_all.sort_values(by='sigma_btl',inplace=True)
time_data_all.sort_values(by='sigma_ctd',inplace=True)
for station in station_list:
    rinko_coef = rinko_coef_df.loc[station].values
    #time_data = time_data_all[time_data_all['SSSCC'].str[0:3] == station].copy()
    time_data = time_data_all[time_data_all['oxy_stn_group'] == station].copy()
    time_data['CTDOXY'] = oxy_fitting.SB_oxy_eq(sbe43_coef_df.loc[station],time_data[dov_col],time_data[p_col],time_data[t_col],time_data['dv_dt'],time_data['OS_ctd'])
    time_data['CTDRINKO'] = rinko.rinko_curve_fit_eq((time_data[p_col],time_data[t_col],time_data[rinko_volts],time_data['OS_ctd']),rinko_coef[0],rinko_coef[1],
                                                     rinko_coef[2],rinko_coef[3],rinko_coef[4],rinko_coef[5],rinko_coef[6],rinko_coef[7])
    
    time_data_all.loc[time_data_all['SSSCC'].str[0:3] == station,'CTDOXY'] = time_data['CTDOXY']
    time_data_all.loc[time_data_all['SSSCC'].str[0:3] == station,'CTDRINKO'] = time_data['CTDRINKO']
#    for stn in time_data['SSSCC'].unique():
#        td = time_data.loc[time_data['SSSCC'] == stn].copy()
#        td.sort_values(by='sigma_ctd',inplace=True)
#        time_data_all.loc[time_data_all['SSSCC'] == stn,'CTDOXY'] = td['CTDOXY']
#        time_data_all.loc[time_data_all['SSSCC'] == stn,'CTDRINKO'] = td['CTDRINKO']
#    print(station + ' done!')
time_data_all['CTDRINKO_FLAG_W'] = 2
time_data_all['CTDOXY_FLAG_W'] = 2

### Calculate Depths

In [ ]:
depth_dict = {}
for station in station_list:
    print(station)
    time_data = time_data_all[time_data_all['SSSCC'].str[0:3] == station].copy()
    max_depth = process_ctd.find_cast_depth(time_data['CTDPRS'],time_data['GPSLAT'],time_data['ALT'])
    depth_dict[station] = max_depth
depth_df = pd.DataFrame.from_dict(depth_dict,orient='index')
depth_df.reset_index(inplace=True)
depth_df.rename(columns={0:'DEPTH', 'index':'STNNBR'}, inplace=True)

depth_df.to_csv('data/logs/depth_log.csv',index=False)

In [ ]:
### Apply Depths to DFs
btl_data_all['DEPTH'] = '-999'
time_data_all['DEPTH'] = '-999'
depth_df = pd.read_csv('data/logs/depth_log.csv')
depth_df.dropna(inplace=True)
#manual_depth_df = pd.read_csv('data/logs/manual_depth_log.csv')
#full_depth_df = pd.concat([depth_df,manual_depth_df])
full_depth_df = depth_df.copy()
full_depth_df['STNNBR'] = full_depth_df['STNNBR'].astype(str)
full_depth_df['STNNBR'] = full_depth_df['STNNBR'].str.pad(width=3,fillchar='0')
for station in station_list:
    print(station)
    btl_data_all.loc[btl_data_all['SSSCC'].str[0:3] == station,'DEPTH'] = full_depth_df.loc[full_depth_df['STNNBR'] == station,'DEPTH'].values[0]
    time_data_all.loc[time_data_all['SSSCC'].str[0:3] == station,'DEPTH'] = full_depth_df.loc[full_depth_df['STNNBR'] == station,'DEPTH'].values[0]

In [ ]:
btl_data_all = process_ctd.merge_cond_flags(btl_data_all,qual_flag_cond, c_btl_col)

btl_data_all = process_ctd.merge_refcond_flags(btl_data_all,qual_flag_cond)

btl_data_all = process_ctd.merged_reftemp_flags(btl_data_all,qual_flag_temp)

btl_data_all = process_ctd.merge_temp_flags(btl_data_all, qual_flag_temp, t_btl_col)

In [ ]:
qual_flag_temp.to_csv('data/logs/qual_flag_temp_new.csv',index=False)
qual_flag_cond.to_csv('data/logs/qual_flag_cond_new.csv',index=False)
#coef_df.to_csv('data/logs/oxygen_coef.csv',index_label=False)

In [ ]:
time_data_all.rename(columns={'FLUOR':'CTDFLUOR','CTDBACKSCATTER':'CTDBBP700RAW'}, inplace=True)

In [ ]:
#Add flags
time_data_all['CTDBBP700RAW_FLAG_W'] = 1
time_data_all['CTDFLUOR_FLAG_W'] = 1
time_data_all['CTDXMISS_FLAG_W'] = 1

btl_data_all['CTDFLUOR'] = btl_data_all['FLUOR']
btl_data_all['CTDFLUOR_FLAG_W'] = 1
btl_data_all['CTDXMISS_FLAG_W'] = 1

In [ ]:
time_data_all.sort_values(by='master_index',inplace=True)
btl_data_all.sort_values(by='master_index', inplace=True)

In [ ]:
time_data_all = process_ctd.format_time_data(time_data_all)
time_export = process_ctd.export_bin_data(time_data_all,ssscc,sample_rate, search_time,p_column_names,ssscc_col='SSSCC')

In [ ]:
btl_data_all = process_ctd.flag_missing_btl_values(btl_data_all,settings.btl_flagged_params)

In [ ]:
btl_export = process_ctd.format_btl_data(btl_data_all,settings.btl_flagged_params)

In [ ]:
process_ctd.export_btl_data(btl_export, expocode, settings.btl_column_names, settings.btl_column_units, sectionID)

### SANITY CHECKS

In [ ]:
len(btl_data_all[btl_data_all['CTDOXY_FLAG_W']==3])

In [ ]:
#OXYGEN
#df_good = all_oxy_df[all_oxy_df['CTDOXY_FLAG_W']==2].copy()
#df_good.loc[df_good['SSSCC_btl'] == '18603','SSSCC_btl'] = '01863'
btl_data_all = oxy_fitting.flag_oxy_data(btl_data_all)
df_good = btl_data_all[btl_data_all['CTDOXY_FLAG_W']==2].copy()
df_good['res'] = df_good['OXYGEN'] - df_good['CTDOXY']
df_good.loc[df_good['SSSCC'] == '18603','SSSCC'] = '01863'

In [ ]:
df_good['res'].std()

In [ ]:
len(btl_data_all[btl_data_all['CTDOXY_FLAG_W']==3])

In [ ]:
def btl_oxy_residuals_pressure_plot(df):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    cm = ax.scatter(df['res'],-df['CTDPRS'], marker='+', c=df['SSSCC'].str[0:3].astype(int), cmap='rainbow')
    ax.set_xlim(-10,10)
    ax.set_title('OXYGEN-CTDOXY vs CTDPRS')
    ax.set_xlabel('CTDOXY Residual (umol/kg)')
    ax.set_ylabel('Pressure (dbar)')
    cbar = fig.colorbar(cm)
    cbar.set_label('Station Number')
    
def btl_c1_residuals_pressure_plot(df):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    cm = ax.scatter(df['BTL_C1'],-df['CTDPRS'], marker='+', c=df['SSSCC'].str[0:3].astype(int), cmap='rainbow')
    ax.set_xlim(-0.02,0.02)
    ax.set_title('BTLCOND-CTDCOND1 vs CTDPRS')
    ax.set_xlabel('C1 Residual (mS/cm)')
    ax.set_ylabel('Pressure (dbar)')
    cbar = fig.colorbar(cm)
    cbar.set_label('Station Number')


def btl_c2_residuals_pressure_plot(df):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    cm = ax.scatter(df['BTL_C2'],-df['CTDPRS'], marker='+', c=df['SSSCC'].str[0:3].astype(int), cmap='rainbow')
    ax.set_xlim(-0.02,0.02)
    ax.set_title('BTLCOND-CTDCOND2 vs CTDPRS')
    ax.set_xlabel('C2 Residual (mS/cm)')
    ax.set_ylabel('Pressure (dbar)')
    cbar = fig.colorbar(cm)
    cbar.set_label('Station Number')


def c1_c2_residuals_pressure_plot(df):
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    cm = ax.scatter(df['C1_C2'],-df['CTDPRS'], marker='+', c=df['SSSCC'].str[0:3].astype(int), cmap='rainbow')
    ax.set_xlim(-0.02,0.02)
    ax.set_title('CTDCOND1-CTDCOND2 vs CTDPRS')
    ax.set_xlabel('C1-C2 Residual (mS/cm)')
    ax.set_ylabel('Pressure (dbar)')
    cbar = fig.colorbar(cm)
    cbar.set_label('Station Number')


In [ ]:
btl_oxy_residuals_pressure_plot(df_good)

In [ ]:
import ctdcal.merge_codes as merge_codes

In [ ]:
btl_data_all['REFTMP'] = btl_data_all['T90']

In [ ]:
btl_data_all['C1_C2'] = btl_data_all['CTDCOND1'] - btl_data_all['CTDCOND2']
btl_data_all['BTLCOND'] = gsw.C_from_SP(btl_data_all['SALNTY'], btl_data_all['CTDTMP1'], btl_data_all['CTDPRS'])
sal_df = btl_data_all[btl_data_all['SALNTY_FLAG_W'] != 9].copy()
sal_df['BTL_C1'] = sal_df['BTLCOND'] - sal_df['CTDCOND1']
sal_df['BTL_C2'] = sal_df['BTLCOND'] - sal_df['CTDCOND2']
sal_df.loc[(sal_df['CTDPRS'] >= 900) & (sal_df['BTL_C2']  <= -0.008),'CTDSAL_FLAG_W'] = 3 
sal_df = sal_df[sal_df['SALNTY_FLAG_W'] == 2]
sal_df = sal_df[sal_df['CTDSAL_FLAG_W'] == 2]
sal_df.loc[sal_df['SSSCC'] == '18603','SSSCC'] = '01863'

In [ ]:
btl_c1_residuals_pressure_plot(sal_df)

In [ ]:
btl_c2_residuals_pressure_plot(sal_df)

In [ ]:
c1_c2_residuals_pressure_plot(sal_df)

In [ ]:
btl_data_all['STNNO'].values

In [ ]:
sal_df.loc[(sal_df['CTDPRS'] >=1000) & (sal_df['BTL_C1'] <= -0.010),['SSSCC','CTDPRS','BTL_C2']]

In [ ]:
sal_df.loc[(sal_df['CTDPRS'] >=900) & (sal_df['BTL_C2'] <= -0.010),['SSSCC','CTDPRS','BTL_C2']]

In [ ]:
#Salinity
print('Fractions of bad salinities by station:')
ssscc.sort()
for station in ssscc:
    btl_data = btl_data_all[btl_data_all['SSSCC'] == station].copy()
    btl_data = btl_data[btl_data['SALNTY_FLAG_W'] != 9]
    per_bad = len(btl_data[btl_data['SALNTY_FLAG_W'] == 3])/len(btl_data)
    print(station,' :',per_bad)
    

In [ ]:
import cmocean
import ctdcal.ctd_plots as ctd_plots

In [ ]:
btl_data_all.loc[btl_data_all['STNNO'] == '186','STNNO'] = '018'
btl_data_all.loc[btl_data_all['STNNO'] == '183','STNNO'] = '018'

In [ ]:
btl_data_all = btl_data_all[btl_data_all['CTDSAL_FLAG_W'] == 2].copy()
btl_data_all = btl_data_all[btl_data_all['CTDOXY_FLAG_W'] == 2].copy()
btl_data_all = btl_data_all[btl_data_all['SALNTY_FLAG_W'] == 2].copy()
btl_data_all = btl_data_all[btl_data_all['REFTMP_FLAG_W'] == 2].copy()
btl_data_all['BTLCOND'] = gsw.C_from_SP(btl_data_all['SALNTY'],btl_data_all['CTDTMP2'],btl_data_all['CTDPRS'])

In [ ]:
btl_data_all['cond2_res'] = btl_data_all['BTLCOND'] - btl_data_all['CTDCOND2']

In [ ]:
btl_data_all = btl_data_all[btl_data_all['cond2_res'].abs() <= 0.025]


In [ ]:
btl_data_all['C1SAL'] = gsw.SP_from_C(btl_data_all['CTDCOND1'],btl_data_all['CTDTMP1'],btl_data_all['CTDPRS'])
btl_data_all['C2SAL'] = gsw.SP_from_C(btl_data_all['CTDCOND2'],btl_data_all['CTDTMP2'],btl_data_all['CTDPRS'])

In [ ]:
btl_data_all['STNNO'] = btl_data_all['STNNO'].astype(int)
ctd_plots.btl_c1_residuals_compare_plot(btl_data_all['BTLCOND'],btl_data_all['CTDCOND1'],btl_data_all['CTDPRS'])
ctd_plots.btl_c1_residuals_pressure_plot(btl_data_all['BTLCOND'], btl_data_all['CTDCOND1'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_c1_residuals_station_deep_plot(btl_data_all['BTLCOND'], btl_data_all['CTDCOND1'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_c1_residuals_station_plot(btl_data_all['BTLCOND'], btl_data_all['CTDCOND1'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_c1_residuals_station_uncorrected_plot(btl_data_all['BTLCOND'], btl_data_all['CTDCOND1'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])

ctd_plots.btl_c2_residuals_compare_plot(btl_data_all['BTLCOND'], btl_data_all['CTDCOND2'], btl_data_all['CTDPRS'])
ctd_plots.btl_c2_residuals_pressure_plot(btl_data_all['BTLCOND'], btl_data_all['CTDCOND2'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_c2_residuals_station_deep_plot(btl_data_all['BTLCOND'], btl_data_all['CTDCOND2'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_c2_residuals_station_plot(btl_data_all['BTLCOND'], btl_data_all['CTDCOND2'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_c2_residuals_station_uncorrected_plot(btl_data_all['BTLCOND'], btl_data_all['CTDCOND2'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])

btl_data_all['C1_C2'] = btl_data_all['CTDCOND1'] - btl_data_all['CTDCOND2']
btl_data_all['T1_T2'] = btl_data_all['CTDTMP1'] - btl_data_all['CTDTMP2']
test = btl_data_all[btl_data_all['C1_C2'] >= -0.021]
ctd_plots.c_t_coherence_plot(test['CTDTMP1'], test['CTDTMP2'], test['CTDCOND1'], test['CTDCOND2'], test['CTDPRS'])
ctd_plots.c1_c2_residuals_compare_plot(btl_data_all['BTLCOND'],btl_data_all['CTDCOND1'], btl_data_all['CTDCOND2'], btl_data_all['CTDPRS'])
ctd_plots.c1_c2_residuals_pressure_plot(btl_data_all['CTDCOND1'], btl_data_all['CTDCOND2'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])
ctd_plots.c1_c2_residuals_station_deep_plot(btl_data_all['CTDCOND1'], btl_data_all['CTDCOND2'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])
ctd_plots.c1_c2_residuals_station_plot(btl_data_all['CTDCOND1'], btl_data_all['CTDCOND2'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])
ctd_plots.c1_c2_residuals_station_uncorrected_plot(btl_data_all['CTDCOND1'], btl_data_all['CTDCOND2'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])

ctd_plots.btl_sal_station_plot(btl_data_all['SALNTY'],btl_data_all['C2SAL'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_sal_station_deep_plot(btl_data_all['SALNTY'],btl_data_all['C2SAL'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_sal_pressure_plot(btl_data_all['SALNTY'],btl_data_all['C2SAL'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])

ctd_plots.btl_t1_residuals_pressure_plot(btl_data_all['T90'], btl_data_all['CTDTMP1'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_t1_residuals_station_deep_plot(btl_data_all['T90'], btl_data_all['CTDTMP1'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_t1_residuals_station_plot(btl_data_all['T90'], btl_data_all['CTDTMP1'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])

ctd_plots.btl_t2_residuals_pressure_plot(btl_data_all['T90'], btl_data_all['CTDTMP2'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_t2_residuals_station_deep_plot(btl_data_all['T90'], btl_data_all['CTDTMP2'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])
ctd_plots.btl_t2_residuals_station_plot(btl_data_all['T90'], btl_data_all['CTDTMP2'],btl_data_all['CTDPRS'],btl_data_all['STNNO'])

ctd_plots.t1_t2_residuals_pressure_plot(btl_data_all['CTDTMP1'], btl_data_all['CTDTMP2'], btl_data_all['CTDPRS'], btl_data_all['STNNO'].astype(int))
ctd_plots.t1_t2_residuals_station_deep_plot(btl_data_all['CTDTMP1'], btl_data_all['CTDTMP2'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])
ctd_plots.t1_t2_residuals_station_plot(btl_data_all['CTDTMP1'], btl_data_all['CTDTMP2'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])

ctd_plots.btl_oxy_residuals_pressure_concentration_plot(btl_data_all['OXYGEN'], btl_data_all['CTDOXY'], btl_data_all['STNNO'])
ctd_plots.btl_oxy_residuals_pressure_plot(btl_data_all['OXYGEN'], btl_data_all['CTDOXY'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])
ctd_plots.btl_oxy_residuals_station_concentration_plot(btl_data_all['OXYGEN'], btl_data_all['CTDOXY'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])
ctd_plots.btl_oxy_residuals_station_deep_plot(btl_data_all['OXYGEN'], btl_data_all['CTDOXY'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])
ctd_plots.btl_oxy_residuals_station_deep_temperature_plot(btl_data_all['OXYGEN'], btl_data_all['CTDOXY'], btl_data_all['CTDTMP1'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])
ctd_plots.btl_oxy_residuals_station_plot(btl_data_all['OXYGEN'], btl_data_all['CTDOXY'], btl_data_all['CTDPRS'], btl_data_all['STNNO'])
ctd_plots.btl_oxy_residuals_station_temperature_plot(btl_data_all['OXYGEN'], btl_data_all['CTDOXY'], btl_data_all['CTDTMP1'], btl_data_all['STNNO'])
ctd_plots.btl_oxy_residuals_temperature_plot(btl_data_all['OXYGEN'], btl_data_all['CTDOXY'], btl_data_all['CTDTMP1'], btl_data_all['STNNO'])



In [ ]:
btl_data_all['C1_C2'] = btl_data_all['CTDCOND1'] - btl_data_all['CTDCOND2']
btl_data_all['T1_T2'] = btl_data_all['CTDTMP1'] - btl_data_all['CTDTMP2']

In [ ]:
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    cm = ax.scatter(test['T1_T2'], test['C1_C2'], marker='+', c=test['CTDPRS'], cmap=plt.cm.viridis_r)

In [ ]:
test[['CTDCOND1','CTDCOND2','BTLCOND']]

In [ ]:
    t1_t2 = t1_vals - t2_vals
    c1_c2 = c1_vals - c2_vals
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    cm = ax.scatter(t1_t2, c1_c2, marker='+', c=press, cmap=plt.cm.viridis_r)
    ax.set_xlim(-0.02,0.02)
    ax.set_title('T1-T2 vs C1-C2')
    ax.set_xlabel('T1-T2 Residual (T90 C)')
    ax.set_ylabel('C1-C2 Residual (mS/cm)')
    cbar = fig.colorbar(cm)
    cbar.set_label('Pressure (dbar)')

